In [ ]:
# pip install httpx websockets

In [ ]:
#!/usr/bin/env python3
import yaml
import httpx
import sys
import json
import asyncio
import websockets
import time


In [ ]:


# Backend 1 URL
backend1_url = "http://localhost:8000/execute"
# WebSocket URL to receive streaming responses
ws_url = "ws://localhost:8000/ws"

# Load the YAML flow file
yaml_file_path = "./execution_flows/simple-ai-flow.yaml"
print(f"Loading flow from {yaml_file_path}...")

with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)


In [ ]:
flow_id = yaml_data.get("flow_id", "flow-" + yaml_file_path.split("/")[-1].split(".")[0])

In [17]:
async def execute_and_stream():
    # Connect to the WebSocket without sending parameters in URL
    websocket_url = f"ws://localhost:8000/ws/execute/{flow_id}"
    
    print(f"Connecting to WebSocket at {websocket_url}")
    
    try:
        async with websockets.connect(websocket_url) as websocket:
            print("Connected. Waiting for ready message...")
            
            # Wait for ready message
            ready_msg = await websocket.recv()
            print(f"Received: {ready_msg}")
            
            # Send flow definition
            flow_definition_str = json.dumps(yaml_data.get("flow_definition"))
            await websocket.send(flow_definition_str)
            print("Sent flow definition")
            
            # Wait for acknowledgment
            ack1 = await websocket.recv()
            print(f"Received: {ack1}")
            
            # Send initial inputs
            initial_inputs_str = json.dumps(yaml_data.get("initial_inputs", {}))
            await websocket.send(initial_inputs_str)
            print("Sent initial inputs")
            
            # Wait for acknowledgment
            ack2 = await websocket.recv()
            print(f"Received: {ack2}")
            
            # Send config (null in this case)
            await websocket.send("null")
            print("Sent null config")
            
            # Wait for final acknowledgment
            ack3 = await websocket.recv()
            print(f"Received: {ack3}")
            
            print("Flow execution starting. Receiving events...")
            
            # Now receive streaming events
            while True:
                try:
                    message = await websocket.recv()
                    
                    try:
                        data = json.loads(message)
                        event_type = data.get("type", "unknown")
                        timestamp = data.get("timestamp", time.time())
                        
                        # Format timestamp for display
                        formatted_time = time.strftime("%H:%M:%S", time.localtime(timestamp))
                        
                        print(f"[{formatted_time}] {event_type}")
                        
                        # For some events, show more detailed information
                        if event_type == "element_started":
                            element_data = data.get("data", {})
                            element_name = element_data.get("element_name", "Unknown")
                            element_type = element_data.get("element_type", "Unknown")
                            print(f"  ▶ Started: {element_name} ({element_type})")
                        
                        elif event_type == "element_completed":
                            element_data = data.get("data", {})
                            element_name = element_data.get("element_name", "Unknown")
                            print(f"  ✓ Completed: {element_name}")
                        
                        elif event_type == "element_error":
                            element_data = data.get("data", {})
                            element_name = element_data.get("element_name", "Unknown")
                            error = element_data.get("error", "Unknown error")
                            print(f"  ✗ Error in {element_name}: {error}")
                        
                        elif event_type == "llm_chunk":
                            chunk = data.get("data", {}).get("content", "")
                            print(f"  📝 LLM: {chunk}", end="", flush=True)
                        
                        elif event_type == "flow_completed":
                            print(f"  ✅ Flow completed")
                            return
                            
                    except json.JSONDecodeError:
                        print(f"Received non-JSON message: {message[:100]}...")
                        
                except websockets.exceptions.ConnectionClosed:
                    print("WebSocket connection closed")
                    break
                    
    except Exception as e:
        print(f"Error in WebSocket connection: {str(e)}")

# Call the function to execute and stream
await execute_and_stream()

Connecting to WebSocket at ws://localhost:8000/ws/execute/simple-ai-flow
Connected. Waiting for ready message...
Received: {"status": "ready", "message": "Send flow_definition as JSON"}
Sent flow definition
Received: {"status": "received_flow", "message": "Send initial_inputs as JSON"}
Sent initial inputs
Received: {"status": "received_inputs", "message": "Send config as JSON or 'null'"}
Sent null config
Received: {"status": "starting", "message": "Starting flow execution"}
Flow execution starting. Receiving events...
WebSocket connection closed
